In [11]:
import pandas as pd

# DATA
DATA_BASE_PATH = "s3://wynk-ml-workspace/projects/_temp/rnn_recommendation/with_metadata/day=2021-05-16to2021-05-22/"
TRAIN_DATA_DIR_PATH = "train/"
VAL_DATA_DIR_PATH = "val/"
TEST_DATA_DIR_PATH = "test/"

# TRAIN DATA
TRAIN_DATA_PATH = (DATA_BASE_PATH + TRAIN_DATA_DIR_PATH + 
                   "SessionizedSongDataPreprocessedPopBias_4_concat_seq_csv/part-00000-5af881ad-6a26-4315-ad13-754b610a7570-c000.csv")
TRAIN_SONGS_INFO_PATH = (DATA_BASE_PATH + TRAIN_DATA_DIR_PATH + 
                         "song2info/part-00000-47cf8333-7ca4-4c7e-9728-060ce46eb410-c000.snappy.parquet")

In [44]:
            train_songs_info_df = pd.read_parquet(TRAIN_SONGS_INFO_PATH, 
                                                  columns=["song_id", "title", "album", "artist", "frequency", "language", "publishedYear"])
            print('train_songs_info_df.shape  : ', train_songs_info_df.shape) #(10906107, 6)

train_songs_info_df.shape  :  (149345, 7)


In [46]:
!python3 dataset.py
# !python3 config.py

2021-06-20 09:00:43.694609: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300065000 Hz
2021-06-20 09:00:43.694826: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f5764d1f20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-06-20 09:00:43.694847: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-06-20 09:00:43.694969: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.

##### Building vocab... #####
song_info_df.shape:  (149345, 3)
song_info_df.columns:  Index(['song_id', 'song_embedding_id', 'frequency'], dtype='object')
self.popular_songs_num:  7467
self.vocab_size:  149347
##### Vocab built... #####


##### Mapping song2info... #####

Mapping song2info...
train_songs_info_df.shape:  (149345, 7)
Traceback (most recent ca